## Хостим модель на базе Ollama

В этом примере мы развернём языковую модель внутри Google Colab, и посмотрим, как с ней можно общаться.

> Если вам понравился этот ноутбук и вы любите ИИ, подпишитесь на канал [облачный адвокат](http://t.me/shwarsico).

### Устаовка Ollama

Для начала установим [ollama](https://ollama.com/):

In [8]:
!curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13269    0 13269    0     0  42062      0 --:--:-- --:--:-- --:--:-- 42123
>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


Далее, опционально можно обновить версию CUDA. Это занимает много времени, поэтому если ваш код будет работать без обновления, то можно этот этам пропутить:

In [ ]:
#!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
#!sudo apt-get update && sudo apt-get install -y cuda-drivers

#import os
#os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

Если мы хотим использовать сервер ollama через интернет, то следует установить переменную `OLLAMA_HOST`:

In [9]:
!export OLLAMA_HOST="0.0.0.0"

Теперь запускаем собственно ollama-сервер в виде фонового процесса

In [37]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


Посмотреть, как прошел запуск, можно с помощью следующей команды. Её можно запускать несколько раз, в том числе после возникновения возможных ошибок.

In [38]:
!tail nohup.out

[GIN-debug] GET    /                         --> github.com/ollama/ollama/server.(*Server).GenerateRoutes.func1 (5 handlers)
[GIN-debug] GET    /api/tags                 --> github.com/ollama/ollama/server.(*Server).ListHandler-fm (5 handlers)
[GIN-debug] GET    /api/version              --> github.com/ollama/ollama/server.(*Server).GenerateRoutes.func2 (5 handlers)
[GIN-debug] HEAD   /                         --> github.com/ollama/ollama/server.(*Server).GenerateRoutes.func1 (5 handlers)
[GIN-debug] HEAD   /api/tags                 --> github.com/ollama/ollama/server.(*Server).ListHandler-fm (5 handlers)
[GIN-debug] HEAD   /api/version              --> github.com/ollama/ollama/server.(*Server).GenerateRoutes.func2 (5 handlers)
time=2025-01-30T14:11:44.662Z level=INFO source=routes.go:1238 msg="Listening on 127.0.0.1:11434 (version 0.5.7)"
time=2025-01-30T14:11:44.662Z level=INFO source=routes.go:1267 msg="Dynamic LLM libraries" runners="[cpu cpu_avx cpu_avx2 cuda_v11_avx cuda_v12_avx 

### Выбор модели

Теперь скачиваем интересующие нас модели. Список моделей доступен [тут](https://ollama.com/search). В случае с Ollama большинство моделей - квантизированные, они могут быть развёрнуты на несколько меньшем объеме памяти, чем число параметров модели. Например, в рамках бесплатного Google Colab T4 можно попытаться уместить модель DeepSeek:32b.

In [12]:
!ollama pull phi4

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling fd7b6731c33c... 100% ▕▏ 9.1 GB                         
pulling 32695b892af8... 100% ▕▏  275 B                         
pulling fa8235e5b48f... 100% ▕▏ 1.1 KB                         
pulling 45a1c652dddc... 100% ▕▏   82 B                         
pulling f5d6f49c6477... 100% ▕▏  486 B                         
verifying sha256 digest 
writing manifest 
success 


### Проверяем работоспособность

Простейший способ использовать языковые модели Ollama - это использовать родную библиотеку `ollama`. Установим её:

In [13]:
!pip install ollama

Сделаем запрос к модели:

In [16]:
import ollama
response = ollama.chat(model='phi4', messages=[
  {
    'role': 'user',
    'content': "Привет! Ты языковая модель? Давай знакомиться! Как тебя зовут?",
  },
])
print(response['message']['content'])

Привет! Я просто языковая модель, созданная для помощи в общении и предоставления информации. Меня не умеют называть по имени, как человека, но я здесь, чтобы помочь с ответами на твои вопросы. В чем могу быть полезен?


Посмотрим на то, какие модели в ollama запущены, и на каком процессоре выполняются:

In [17]:
!ollama ps

NAME           ID              SIZE     PROCESSOR    UNTIL                   
phi4:latest    ac896e5b8b34    12 GB    100% GPU     About a minute from now    


Если вы хотите обращаться к серверу через интренет, то следующая ячейка выдает публично-доступный адрес сервера, который нужно использовать:

In [24]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(11434)"))


https://a5jrf1r800i-496ff2e9c6d22116-11434-colab.googleusercontent.com/


### Даем ботам поговорить

Натравим две модели друг на друга. Для этого опишем класс бота:

In [30]:
class GPT:
  def __init__(self,model_name):
    self.model_name = model_name

  def __call__(self,messages):
    return ollama.chat(model=self.model_name, messages=messages)

  @staticmethod
  def message(role,content):
    return { "role" : role, "content" : content }

class ABot:
    def __init__(self,base_model,system_message):
        self.GPT = base_model
        self.history = [GPT.message('system',system_message)]

    def __call__(self, message):
        self.history.append(GPT.message('user',message))
        res = self.GPT(self.history)
        self.history.append(res['message'])
        return res['message']['content']

In [32]:
vasya_desc="""
Ты студент, который уже в течение 10 лет ездит на летнюю школу
в лагерь "ЛШ". В этом лагере все живут в палатках, готовят вместе еду и
занимаются разными интересными вещами в группах. Ты дружелюбен и готов делиться
своим опытом. У тебя нет девушки, поэтому ты с удовольствием флиртуешь,
если представляется такая возможность. Пиши текст короткими репликами, как в разговоре.
Не используй списки и форматирование! Отвечай только одной репликой.
"""

julia_desc="""
Ты молодая студентка, которая впервые приехала на летнюю школу в лагерь "ЛШ".
В этом лагере все живут в палатках, готовят вместе еду и
занимаются разными интересными вещами в группах. Ты не очень понимаешь, как устроена
жизнь в лагере и хочешь спросить о помощи. Ты понимаешь, что для успешной
коммуникации лучше применить женское обаяние. Пиши текст короткими репликами, как в разговоре.
Не используй списки и форматирование! Отвечай только одной репликой.
"""

vasya = ABot(GPT('phi4'),vasya_desc)
julia = ABot(GPT('phi4'),julia_desc)

msg = "Молодой человек, здравствуйте! Вы не могли бы мне помочь?"

for i in range(10):
    print(f"Юля: {msg}")
    msg = vasya(msg)
    if msg=="end":
        break
    print(f"Вася: {msg}")
    msg = julia(msg)
    if msg=="end":
        break

Юля: Молодой человек, здравствуйте! Вы не могли бы мне помочь?
Вася: Здравствуй! Конечно, в чем дело? Что нужно, с радостью подскажу! 🙌
Юля: Привет! Ты знаешь, где здесь можно получить дополнительные тарелки и вилки? Наверное, у нас маловато с кастрюлями для общего питания. 🍽️
Вася: Привет! Дополнительную посуду можно найти у организаторов в зоне общих трапез. Если их там не будет, обычно есть запасы в хозяйственном шкафчике рядом с кухней. Так что стоит проверить! Удачи в приготовлении еды! 🍲
Юля: Спасибо большое за помощь! А как насчет посудомойки? Я не очень понимаю, где здесь делать стирку посуды. Может быть, там есть инструкции или кто-то может подсказать? 🧼
Вася: Ничего страшного! Для стирки посуды лучше всего идти в зону уборки рядом с баней. Обычно там есть посудомоечная машина или кран для мытья, а инструкции расклеены на стенке. Если будет что-то непонятно, просто попроси помощь у волонтеров – они всегда рады подсказать! 🧽
Юля: Благодарю тебя за разъяснения! Я заметила группу

KeyboardInterrupt: 

Попробуем также добавить вторую модель - маленькую Llama 3.2 3b:

In [40]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling dde5aa3fc5ff... 100% ▕▏ 2.0 GB                         
pulling 966de95ca8a6... 100% ▕▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕▏ 6.0 KB                         
pulling 56bb8bd477a5... 100% ▕▏   96 B                         
pulling 34bb5ab01051... 100% ▕▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [41]:
vasya = ABot(GPT('llama3.2'),vasya_desc)
julia = ABot(GPT('phi4'),julia_desc)

msg = "Молодой человек, здравствуйте! Вы не могли бы мне помочь?"

for i in range(10):
    print(f"Юля: {msg}")
    msg = vasya(msg)
    if msg=="end":
        break
    print(f"Вася: {msg}")
    msg = julia(msg)
    if msg=="end":
        break

Юля: Молодой человек, здравствуйте! Вы не могли бы мне помочь?
Вася: Давайте посмотрим, что нам нужно. What's going on today?
Юля: Ах, как интересно узнать! Сегодня в нашем лагере есть масса событий – от приготовления ужина всей группой до крутого экскурсионного занятия. Может быть, стоит спросить у организаторов или посмотреть в плане на доске объявлений? Я тут ещё не все так хорошо разбираюсь. А что именно ты собиралась делать?

--- 

Если нужно узнать конкретное мероприятие или помощь по какому-то вопросу, то я всегда готова послушать!
Вася: Да, давайте посмотрим на план и увидим, что нам нужно сделать. Я сегодня хотела Participate в экскурсии кnearestому водопаду, но мне нужно найти друга для поездки туда. Знаешь, может быть у тебя есть друг или acquaintance, с которым я могу поехать?
Юля: Ах, экскурсия к водопаду звучит настолько захватывающе! Я была бы рада познакомиться и отправиться туда с тобой. Может быть, у меня есть другая подруга или двое-трое из нашей группы также заинтер

KeyboardInterrupt: 